# Teifi catchment
# 3. GRU

# Hyperparameter tuning using model_builder function from tensorflow   (Standard normalization)
(After midterm)
#(https://www.tensorflow.org/tutorials/keras/keras_tuner)

In [148]:
import numpy as np
import pandas as pd

### define file path
path = 'Teifi_R and Q with lag.csv'  ####### This excel data is not scaled/normalized

## Read CSV file and convert it to dataframe

df = pd.read_csv( path, sep = ',', header = 'infer')


In [149]:
df

,Date,Rt,R_t_1,R_t_2,R_t_3,R_t_4,R_t_5,R_t_6,R_t_7,R_t_8,Qt
0,09/10/2000,15.7,11.8,1.6,2.8,1.1,7.7,7.7,2.9,12.9,70.30
1,10/10/2000,21.4,15.7,11.8,1.6,2.8,1.1,7.7,7.7,2.9,73.51
2,11/10/2000,10.3,21.4,15.7,11.8,1.6,2.8,1.1,7.7,7.7,96.17
3,12/10/2000,21.9,10.3,21.4,15.7,11.8,1.6,2.8,1.1,7.7,132.60
4,13/10/2000,2.7,21.9,10.3,21.4,15.7,11.8,1.6,2.8,1.1,119.70
...,...,...,...,...,...,...,...,...,...,...,...
3639,26/09/2010,0.0,0.0,0.3,9.5,17.1,0.0,0.1,0.9,0.5,17.29
3640,27/09/2010,0.0,0.0,0.0,0.3,9.5,17.1,0.0,0.1,0.9,15.30
3641,28/09/2010,14.1,0.0,0.0,0.0,0.3,9.5,17.1,0.0,0.1,14.28
3642,29/09/2010,0.2,14.1,0.0,0.0,0.0,0.3,9.5,17.1,0.0,23.13


In [150]:
df = df.set_index('Date')
print(df)

              Rt  R_t_1  R_t_2  R_t_3  R_t_4  R_t_5  R_t_6  R_t_7  R_t_8  \
Date                                                                       
09/10/2000  15.7   11.8    1.6    2.8    1.1    7.7    7.7    2.9   12.9   
10/10/2000  21.4   15.7   11.8    1.6    2.8    1.1    7.7    7.7    2.9   
11/10/2000  10.3   21.4   15.7   11.8    1.6    2.8    1.1    7.7    7.7   
12/10/2000  21.9   10.3   21.4   15.7   11.8    1.6    2.8    1.1    7.7   
13/10/2000   2.7   21.9   10.3   21.4   15.7   11.8    1.6    2.8    1.1   
...          ...    ...    ...    ...    ...    ...    ...    ...    ...   
26/09/2010   0.0    0.0    0.3    9.5   17.1    0.0    0.1    0.9    0.5   
27/09/2010   0.0    0.0    0.0    0.3    9.5   17.1    0.0    0.1    0.9   
28/09/2010  14.1    0.0    0.0    0.0    0.3    9.5   17.1    0.0    0.1   
29/09/2010   0.2   14.1    0.0    0.0    0.0    0.3    9.5   17.1    0.0   
30/09/2010  13.3    0.2   14.1    0.0    0.0    0.0    0.3    9.5   17.1   

           

In [151]:
df.describe()

,Rt,R_t_1,R_t_2,R_t_3,R_t_4,R_t_5,R_t_6,R_t_7,R_t_8,Qt
count,3644.000000,3644.000000,3644.000000,3644.000000,3644.000000,3644.000000,3644.000000,3644.000000,3644.000000,3644.000000
mean,3.992179,3.991767,3.992151,3.989050,3.989352,3.991465,3.993578,3.994292,3.995225,30.440950
std,6.542103,6.541564,6.541383,6.539268,6.539109,6.539064,6.539018,6.538756,6.539785,34.149581
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.085000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.120500
50%,1.100000,1.100000,1.100000,1.100000,1.100000,1.100000,1.100000,1.100000,1.100000,18.020000
75%,5.300000,5.300000,5.300000,5.300000,5.300000,5.300000,5.300000,5.300000,5.300000,40.450000
max,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,373.000000


In [152]:
#### visualize the distribution
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
# Plot the distribution of rainfall data as a histogram
sns.histplot(df["Rt"], kde=False)

# Add labels and title to the plot
plt.xlabel("Rainfall (mm)")
plt.ylabel("Frequency")
plt.title("Histogram of Rainfall Data")
# plt.xlim(0, 80)
# plt.ylim(0, 300)
plt.show()

# 1. Data splitting

##  Train, Validation and Test data 
###  70% train (2550 data points)
###  15% train (547 data points)
###  15% train (547 data points)

In [153]:
df_train = df.iloc[:2550]

df_val = df.iloc[2550:3097] 

# df_test = df.iloc[3097:] 
df_test = df.tail(547).iloc[:]   ## gives the same result as the above line of code

In [154]:
df_train

,Rt,R_t_1,R_t_2,R_t_3,R_t_4,R_t_5,R_t_6,R_t_7,R_t_8,Qt
Date,,,,,,,,,,
09/10/2000,15.7,11.8,1.6,2.8,1.1,7.7,7.7,2.9,12.9,70.300
10/10/2000,21.4,15.7,11.8,1.6,2.8,1.1,7.7,7.7,2.9,73.510
11/10/2000,10.3,21.4,15.7,11.8,1.6,2.8,1.1,7.7,7.7,96.170
12/10/2000,21.9,10.3,21.4,15.7,11.8,1.6,2.8,1.1,7.7,132.600
13/10/2000,2.7,21.9,10.3,21.4,15.7,11.8,1.6,2.8,1.1,119.700
...,...,...,...,...,...,...,...,...,...,...
28/09/2007,0.5,0.0,0.7,7.5,5.6,26.1,2.4,6.0,4.9,15.180
29/09/2007,0.5,0.5,0.0,0.7,7.5,5.6,26.1,2.4,6.0,12.430
30/09/2007,1.1,0.5,0.5,0.0,0.7,7.5,5.6,26.1,2.4,10.660


# 2. Normalization

In [155]:
import pandas as pd
import numpy as np

### General function for all the normalization methods

def normalize_df(data, norm_method):     

    ## Normalize data using specified method
 
    if norm_method == 'minmax':
        # Min-Max normalization
        data_n = (data - data.min()) / (data.max() - data.min())
    elif norm_method == 'zscore':
        # Z-score normalization
        mean = data.mean()
        std = data.std()
        data_n = (data - mean) / std
    elif norm_method == 'max':
        # Max normalization
        data_n = 1 - ((data.max() - data)/data.max())
    else:
        raise ValueError("Invalid normalization method. Choose from: 'minmax', 'zscore', 'max'")
    return data_n



### Standard normalization  (Data sets for training, validation and testing are nromalized separately)

In [156]:
## Z_score(standard)
df_train_norm = normalize_df(df_train, 'zscore')
df_val_norm = normalize_df(df_val, 'zscore')
df_test_norm = normalize_df(df_test, 'zscore')

In [157]:
###example
df_train_norm.describe()

,Rt,R_t_1,R_t_2,R_t_3,R_t_4,R_t_5,R_t_6,R_t_7,R_t_8,Qt
count,2.550000e+03,2.550000e+03,2.550000e+03,2.550000e+03,2.550000e+03,2.550000e+03,2.550000e+03,2.550000e+03,2.550000e+03,2.550000e+03
mean,-2.099671e-15,-2.549943e-15,-1.856685e-15,-2.574847e-15,-2.324807e-15,-2.219924e-15,-2.402566e-15,-2.379970e-15,-2.489621e-15,1.455045e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-6.082633e-01,-6.088429e-01,-6.089583e-01,-6.090787e-01,-6.091250e-01,-6.095501e-01,-6.100155e-01,-6.101733e-01,-6.103107e-01,-8.089168e-01
25%,-6.082633e-01,-6.088429e-01,-6.089583e-01,-6.090787e-01,-6.091250e-01,-6.095501e-01,-6.100155e-01,-6.101733e-01,-6.103107e-01,-6.712789e-01
50%,-4.413379e-01,-4.343651e-01,-4.268866e-01,-4.270013e-01,-4.270444e-01,-4.274705e-01,-4.279337e-01,-4.280835e-01,-4.282758e-01,-3.507176e-01
75%,1.808389e-01,1.801004e-01,1.800189e-01,1.799234e-01,1.798909e-01,1.794615e-01,1.903855e-01,1.902630e-01,1.898844e-01,2.884958e-01
max,8.496762e+00,8.494350e+00,8.494624e+00,8.494792e+00,8.494905e+00,8.494430e+00,8.494071e+00,8.494315e+00,8.491435e+00,9.760296e+00


In [158]:
### convert the dataframes to numpy arrays

df_train_f = df_train_norm.to_numpy()

df_val_f = df_val_norm.to_numpy()

df_test_f = df_test_norm.to_numpy()

In [159]:
### Define input and output sequences (list of sub sequences)

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
    # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1] 
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps_in, n_steps_out = 1, 1    #### 1 input sequences and 1 output sequences



### Split into input and target variables

In [160]:
#### Training data


X_train, y_train = split_sequences(df_train_f, n_steps_in, n_steps_out)
print(X_train.shape, y_train.shape)

# reshape from [samples, timesteps] into [samples, timesteps, features]

n_features = 9   ## 9 rainfall inputs

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
print(X_train.shape)


(2550, 1, 9) (2550, 1)
(2550, 1, 9)


In [161]:
#### validation data

X_val, y_val = split_sequences(df_val_f , n_steps_in, n_steps_out)


X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], n_features))

### reshape

X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], n_features))

print(X_val.shape, y_val.shape)


(547, 1, 9) (547, 1)


In [162]:
#### Test data

X_test, y_test = split_sequences(df_test_f , n_steps_in, n_steps_out)

X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

print(X_test.shape, y_test.shape)


(547, 1, 9) (547, 1)


In [163]:
#### model fitting
import keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import save_model

from tensorflow.keras import initializers

from tensorflow.keras import layers


## Hyperparameter tuning using KerasTuner

In [164]:

import keras_tuner as kt


from keras_tuner.engine.hyperparameters import HyperParameters
from keras_tuner.engine.hyperparameters import Choice,Float

### a. Define model (Using model_buider function)
### (https://www.tensorflow.org/tutorials/keras/keras_tuner)
## GRU

In [165]:
def model_builder(hp):
    model = keras.Sequential()
    ##1. Tune number of units
    hp_units = hp.Int('units', min_value = 1, max_value = 10, step = 1)
    
    ##2. Tune weight initialization method
    
    hp_initializer=hp.Choice('weight_init', ['he_uniform', 'glorot_uniform', 'random_normal' , 'lecun_normal'])
    ### Fix the seeds for reproducibility purposes     
    if hp_initializer == 'he_uniform':
        init = tf.keras.initializers.he_uniform(seed=1)
    elif hp_initializer == 'glorot_uniform':
        init = tf.keras.initializers.glorot_uniform(seed=1)
    elif hp_initializer == 'random_normal':
        init = tf.keras.initializers.RandomNormal(seed=1)
    else:
        init = tf.keras.initializers.lecun_normal(seed=1)

    ##3. Tune number of layers   
    ###4. Tune the activation function

    hp_activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])
    
    for i in range(hp.Int("num_layers", 1, 3)):
        
        model.add(keras.layers.GRU(units=hp_units,
                                activation = hp_activation,
                                return_sequences=True,
                                kernel_initializer = init))
        ##5. Tune dropout rate
        model.add(Dropout(hp.Float('dropout_rate', 0, 0.3, step=0.1)))
        
    
    model.add(keras.layers.Dense(1, kernel_initializer = init))
    model.add(Dropout(hp.Float('dropout_rate', 0, 0.3, step=0.1)))
    
    ##6. Tune optimizer
    
    hp_optimizer = hp.Choice("optimizer", ['Adam', 'SGD', 'RMSprop'])

    model.compile (optimizer = hp_optimizer, 
                   loss='mean_squared_error',
                   metrics=[tf.keras.metrics.MeanSquaredError()])
    
    return model

    
 

### b.  Instantiate the tuner and perform hypertuning

In [166]:
#### Hyperband tuner ( it is a class)

tuner = kt.Hyperband(model_builder,
                     
                     objective= 'mean_squared_error',  ### should be the same as the loss funtion used when compiling
                     
                     max_epochs = 20,
                     
                     factor = 2,  
                     
                     seed = 10,
                     
                     directory='Exhaustive_Opt_norm',
                     
                     project_name='Teifi_GR_test_2')

In [167]:
### Create a callback to stop training early after reaching a certain value for the validation loss.

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [168]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [169]:
from keras.callbacks import TensorBoard

import datetime, os
log_dir = "Exhaustive_Opt_norm/Teifi_GR_test_1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### C. Run hyperparameter search 

# In [170]:
tuner.search(X_train, y_train, 
             epochs=100, 
             validation_data=(X_val, y_val), 
             callbacks=[stop_early,tb])

Trial 92 Complete [00h 00m 20s]
mean_squared_error: 0.4657137989997864

Best mean_squared_error So Far: 0.17782054841518402
Total elapsed time: 00h 21m 52s
INFO:tensorflow:Oracle triggered exit


In [171]:
tuner.results_summary()

Results summary
Results in Exhaustive_Opt_norm\Teifi_GR_test_2
Showing 10 best trials
Trial summary
Hyperparameters:
units: 7
weight_init: random_normal
activation: relu
num_layers: 2
dropout_rate: 0.0
optimizer: Adam
tuner/epochs: 20
tuner/initial_epoch: 10
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0058
Score: 0.17782054841518402
Trial summary
Hyperparameters:
units: 7
weight_init: random_normal
activation: relu
num_layers: 2
dropout_rate: 0.0
optimizer: Adam
tuner/epochs: 10
tuner/initial_epoch: 5
tuner/bracket: 3
tuner/round: 2
tuner/trial_id: 0052
Score: 0.19472935795783997
Trial summary
Hyperparameters:
units: 2
weight_init: random_normal
activation: relu
num_layers: 2
dropout_rate: 0.0
optimizer: Adam
tuner/epochs: 20
tuner/initial_epoch: 10
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0059
Score: 0.19539307057857513
Trial summary
Hyperparameters:
units: 8
weight_init: glorot_uniform
activation: relu
num_layers: 3
dropout_rate: 0.0
optimizer: SGD
tuner/epochs: 20
tuner/i

In [172]:
%tensorboard --logdir Exhaustive_Opt_norm/Teifi_test_1

Reusing TensorBoard on port 6006 (pid 9648), started 1 day, 1:30:03 ago. (Use '!kill 9648' to kill it.)

### steps in tensorboard

In [173]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [174]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [175]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5332), started 23:40:35 ago. (Use '!kill 5332' to kill it.)

In [176]:
%tensorboard --logdir /Exhaustive_Opt_norm/

Reusing TensorBoard on port 6006 (pid 10448), started 23:40:02 ago. (Use '!kill 10448' to kill it.)

In [177]:
%tensorboard --logdir AMT_Exhaustive_optimization/Test1--port=7007

Reusing TensorBoard on port 6006 (pid 10312), started 23:39:18 ago. (Use '!kill 10312' to kill it.)

## d.  get the optimal hyperparameters

In [178]:

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. 

The optimal number of units in the first densely-connected layer is {best_hps.get('units')}, 
The optimal activation function is {best_hps.get('activation')},
The optimal weight initialization is {best_hps.get('weight_init')}
The optimal optimizer is {best_hps.get('optimizer')}.
The optimal number of layers is {best_hps.get('num_layers')},
The optimal dropout rate is {best_hps.get('dropout_rate')},

""")




The hyperparameter search is complete. 

The optimal number of units in the first densely-connected layer is 7, 
The optimal activation function is relu,
The optimal weight initialization is random_normal
The optimal optimizer is Adam.
The optimal number of layers is 2,
The optimal dropout rate is 0.0,




## e. Train the model using the best model

In [181]:
model = tuner.hypermodel.build(best_hps)


In [182]:
### model training 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
GRU_T_1 = model.fit(X_train, y_train, epochs=500, validation_data = (X_val, y_val), shuffle = 0, callbacks=[early_stopping])


Epoch 1/500
80/80 [==============================] - 9s 19ms/step - loss: 0.9986 - mean_squared_error: 0.9986 - val_loss: 0.9887 - val_mean_squared_error: 0.9887
Epoch 2/500
80/80 [==============================] - 1s 8ms/step - loss: 0.9560 - mean_squared_error: 0.9560 - val_loss: 0.8556 - val_mean_squared_error: 0.8556
Epoch 3/500
80/80 [==============================] - 1s 8ms/step - loss: 0.6836 - mean_squared_error: 0.6836 - val_loss: 0.4291 - val_mean_squared_error: 0.4291
Epoch 4/500
80/80 [==============================] - 1s 8ms/step - loss: 0.3409 - mean_squared_error: 0.3409 - val_loss: 0.2190 - val_mean_squared_error: 0.2190
Epoch 5/500
80/80 [==============================] - 1s 9ms/step - loss: 0.2573 - mean_squared_error: 0.2573 - val_loss: 0.1889 - val_mean_squared_error: 0.1889
Epoch 6/500
80/80 [==============================] - 1s 8ms/step - loss: 0.2368 - mean_squared_error: 0.2368 - val_loss: 0.1768 - val_mean_squared_error: 0.1768
Epoch 7/500
80/80 [==============

## Plot training and validation errors

In [183]:
plt.plot(GRU_T_1.history['mean_squared_error'], label = 'Training loss')
plt.plot(GRU_T_1.history['val_mean_squared_error'], label = 'Validation loss')
plt.title('Mean Squared Error')
plt.xlabel('epochs')
plt.ylabel('mse')
plt.legend()

In [184]:
model.summary()   #### this shows us that the number of layers are 3 in this MLP model

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 1, 7)              378       
                                                                 
 dropout_5 (Dropout)         (None, 1, 7)              0         
                                                                 
 gru_4 (GRU)                 (None, 1, 7)              336       
                                                                 
 dropout_6 (Dropout)         (None, 1, 7)              0         
                                                                 
 dense_2 (Dense)             (None, 1, 1)              8         
                                                                 
 dropout_7 (Dropout)         (None, 1, 1)              0         
                                                                 
Total params: 722
Trainable params: 722
Non-trainable 

## f. Test best model

In [185]:
eval_result = model.evaluate(X_test, y_test)
# eval_result = hypermodel.evaluate(X_test, y_test)

print("[test loss, test accuracy]:", eval_result)

18/18 [==============================] - 0s 5ms/step - loss: 0.2390 - mean_squared_error: 0.2390
[test loss, test accuracy]: [0.23897196352481842, 0.23897196352481842]


In [186]:
Y_pred = model.predict(X_test)   ### when using early stopping
# Y_pred = hypermodel.predict(X_test)   

18/18 [==============================] - 1s 6ms/step


In [187]:
Y_pred

array([[[-4.14363056e-01]],

       [[-4.96613532e-01]],

       [[-4.81901199e-01]],

       [[-5.91593027e-01]],

       [[-5.94566762e-01]],

       [[-5.23485303e-01]],

       [[-4.47345942e-01]],

       [[-3.21752876e-01]],

       [[-2.35417396e-01]],

       [[-3.27116191e-01]],

       [[-3.58401150e-01]],

       [[-3.22381824e-01]],

       [[-2.74465829e-01]],

       [[-3.13737959e-01]],

       [[-4.31999952e-01]],

       [[-4.04324442e-01]],

       [[-5.06426990e-01]],

       [[-5.06104290e-01]],

       [[-5.04054725e-01]],

       [[-4.89910215e-01]],

       [[-5.07108986e-01]],

       [[-5.06202936e-01]],

       [[-4.90051687e-01]],

       [[-6.00609362e-01]],

       [[-5.39789796e-01]],

       [[-3.19114268e-01]],

       [[-3.23441774e-01]],

       [[-1.12578198e-01]],

       [[ 2.25894339e-02]],

       [[-5.53055294e-02]],

       [[-1.51854195e-02]],

       [[-5.98950945e-02]],

       [[-1.37374267e-01]],

       [[-3.63355994e-01]],

       [[-3.54

In [188]:
y_test

array([[-0.56212305],
       [-0.57290687],
       [-0.57818475],
       [-0.5882844 ],
       [-0.58369069],
       [-0.51338408],
       [-0.42525648],
       [-0.4581618 ],
       [-0.44578158],
       [-0.46532929],
       [-0.51706557],
       [-0.53729745],
       [-0.54394367],
       [-0.55342431],
       [-0.53309469],
       [-0.49986358],
       [-0.49627984],
       [-0.54475816],
       [-0.56886701],
       [-0.58202913],
       [-0.58880567],
       [-0.59567995],
       [-0.60297776],
       [-0.59861211],
       [-0.57404715],
       [-0.34380768],
       [-0.40114764],
       [-0.43991727],
       [-0.2613815 ],
       [-0.3356628 ],
       [-0.41352786],
       [-0.44610738],
       [-0.46044236],
       [-0.47640633],
       [-0.48878655],
       [-0.4988862 ],
       [-0.49953779],
       [-0.52403759],
       [-0.54381336],
       [-0.56495746],
       [-0.5844726 ],
       [-0.59056497],
       [-0.56036375],
       [ 0.07842289],
       [ 0.11784411],
       [ 0

In [189]:
y_test.shape

(547, 1)

In [190]:
print(y_test[:,:-1].shape)   
print(y_test[:,1:2].shape)

(547, 0)
(547, 0)


In [191]:
y_test[:,-1:]    ##### all rows and all columns including -1(which is the last column)

array([[-0.56212305],
       [-0.57290687],
       [-0.57818475],
       [-0.5882844 ],
       [-0.58369069],
       [-0.51338408],
       [-0.42525648],
       [-0.4581618 ],
       [-0.44578158],
       [-0.46532929],
       [-0.51706557],
       [-0.53729745],
       [-0.54394367],
       [-0.55342431],
       [-0.53309469],
       [-0.49986358],
       [-0.49627984],
       [-0.54475816],
       [-0.56886701],
       [-0.58202913],
       [-0.58880567],
       [-0.59567995],
       [-0.60297776],
       [-0.59861211],
       [-0.57404715],
       [-0.34380768],
       [-0.40114764],
       [-0.43991727],
       [-0.2613815 ],
       [-0.3356628 ],
       [-0.41352786],
       [-0.44610738],
       [-0.46044236],
       [-0.47640633],
       [-0.48878655],
       [-0.4988862 ],
       [-0.49953779],
       [-0.52403759],
       [-0.54381336],
       [-0.56495746],
       [-0.5844726 ],
       [-0.59056497],
       [-0.56036375],
       [ 0.07842289],
       [ 0.11784411],
       [ 0

## Evaluation metics

In [192]:
Y_pred = Y_pred.reshape(547,1)
print(Y_pred)

[[-4.14363056e-01]
 [-4.96613532e-01]
 [-4.81901199e-01]
 [-5.91593027e-01]
 [-5.94566762e-01]
 [-5.23485303e-01]
 [-4.47345942e-01]
 [-3.21752876e-01]
 [-2.35417396e-01]
 [-3.27116191e-01]
 [-3.58401150e-01]
 [-3.22381824e-01]
 [-2.74465829e-01]
 [-3.13737959e-01]
 [-4.31999952e-01]
 [-4.04324442e-01]
 [-5.06426990e-01]
 [-5.06104290e-01]
 [-5.04054725e-01]
 [-4.89910215e-01]
 [-5.07108986e-01]
 [-5.06202936e-01]
 [-4.90051687e-01]
 [-6.00609362e-01]
 [-5.39789796e-01]
 [-3.19114268e-01]
 [-3.23441774e-01]
 [-1.12578198e-01]
 [ 2.25894339e-02]
 [-5.53055294e-02]
 [-1.51854195e-02]
 [-5.98950945e-02]
 [-1.37374267e-01]
 [-3.63355994e-01]
 [-3.54249030e-01]
 [-3.78998011e-01]
 [-4.86840844e-01]
 [-5.06314099e-01]
 [-5.30090034e-01]
 [-5.61089516e-01]
 [-5.45390368e-01]
 [-5.53420067e-01]
 [-4.96510029e-01]
 [-6.12687431e-02]
 [ 5.11240304e-01]
 [ 1.00277913e+00]
 [ 1.57476366e+00]
 [ 1.61241376e+00]
 [ 1.16359627e+00]
 [ 9.25452054e-01]
 [ 5.43980896e-01]
 [ 1.96967095e-01]
 [-2.9517706

In [193]:
y_test 

array([[-0.56212305],
       [-0.57290687],
       [-0.57818475],
       [-0.5882844 ],
       [-0.58369069],
       [-0.51338408],
       [-0.42525648],
       [-0.4581618 ],
       [-0.44578158],
       [-0.46532929],
       [-0.51706557],
       [-0.53729745],
       [-0.54394367],
       [-0.55342431],
       [-0.53309469],
       [-0.49986358],
       [-0.49627984],
       [-0.54475816],
       [-0.56886701],
       [-0.58202913],
       [-0.58880567],
       [-0.59567995],
       [-0.60297776],
       [-0.59861211],
       [-0.57404715],
       [-0.34380768],
       [-0.40114764],
       [-0.43991727],
       [-0.2613815 ],
       [-0.3356628 ],
       [-0.41352786],
       [-0.44610738],
       [-0.46044236],
       [-0.47640633],
       [-0.48878655],
       [-0.4988862 ],
       [-0.49953779],
       [-0.52403759],
       [-0.54381336],
       [-0.56495746],
       [-0.5844726 ],
       [-0.59056497],
       [-0.56036375],
       [ 0.07842289],
       [ 0.11784411],
       [ 0

In [194]:
###nse

import hydroeval as he
nse = he.evaluator(he.nse, Y_pred, y_test )
print(nse)

[0.76059034]


In [195]:
##rmse
rmse = he.evaluator(he.rmse, Y_pred, y_test )
print(rmse)

[0.4888476]


In [196]:
###logrmse
from sklearn.metrics import mean_squared_error
#### log of predicted values
y_pred_lg = np.log1p(Y_pred)

#### log of test values
y_test_lg = np.log1p(y_test)


In [197]:
##calculate mse
mse = mean_squared_error(y_pred_lg, y_test_lg)
rmse_lg = np.sqrt(mse)   
print(rmse_lg)

0.42106193262952407


## g. plot

In [198]:
import matplotlib.pyplot as plt
# plt.plot(Q_pred)
# plt.plot(y_test)

plt.figure(figsize = (16, 12))

plt.subplot(2, 1, 1)

plt.plot(Y_pred[:,-1],'b', label = "Predicted")

plt.plot(y_test[:,-1], 'm',  label = "Observed")  

plt.legend()
plt.grid()

plt.subplot(2,1,2)

plt.plot(Y_pred[:,-1] - y_test[:,-1],'r', label = "Error")

plt.legend()

plt.grid()

##  save_model

## h. Denormalize the data

In [199]:
#### Original data
df_train_norm

# df_val_f

# df_test_f

,Rt,R_t_1,R_t_2,R_t_3,R_t_4,R_t_5,R_t_6,R_t_7,R_t_8,Qt
Date,,,,,,,,,,
09/10/2000,1.774218,1.181452,-0.366196,-0.184231,-0.442218,0.558794,0.558342,-0.170123,1.346565,1.134868
10/10/2000,2.639196,1.773159,1.181413,-0.366309,-0.184270,-0.442644,0.558342,0.558236,-0.170393,1.226337
11/10/2000,0.954766,2.637962,1.773146,1.181349,-0.366351,-0.184698,-0.443107,0.558236,0.557747,1.872033
12/10/2000,2.715071,0.953872,2.637986,1.773101,1.181334,-0.366777,-0.185158,-0.443258,0.557747,2.910104
13/10/2000,-0.198537,2.713822,0.953823,2.637969,1.773096,1.180899,-0.367240,-0.185297,-0.443445,2.542519
...,...,...,...,...,...,...,...,...,...,...
28/09/2007,-0.532388,-0.608843,-0.502750,0.528905,0.240584,3.350681,-0.245852,0.300276,0.132999,-0.435775
29/09/2007,-0.532388,-0.532983,-0.608958,-0.502867,0.528879,0.240155,3.350262,-0.245994,0.299864,-0.514136
30/09/2007,-0.441338,-0.532983,-0.533095,-0.609079,-0.502911,0.528447,0.239699,3.350279,-0.246241,-0.564572


In [200]:
### normalized data after model training
Y_pred    #### model out put
# y_obser = y_test[:,-1:]      ## observed discharge

array([[-4.14363056e-01],
       [-4.96613532e-01],
       [-4.81901199e-01],
       [-5.91593027e-01],
       [-5.94566762e-01],
       [-5.23485303e-01],
       [-4.47345942e-01],
       [-3.21752876e-01],
       [-2.35417396e-01],
       [-3.27116191e-01],
       [-3.58401150e-01],
       [-3.22381824e-01],
       [-2.74465829e-01],
       [-3.13737959e-01],
       [-4.31999952e-01],
       [-4.04324442e-01],
       [-5.06426990e-01],
       [-5.06104290e-01],
       [-5.04054725e-01],
       [-4.89910215e-01],
       [-5.07108986e-01],
       [-5.06202936e-01],
       [-4.90051687e-01],
       [-6.00609362e-01],
       [-5.39789796e-01],
       [-3.19114268e-01],
       [-3.23441774e-01],
       [-1.12578198e-01],
       [ 2.25894339e-02],
       [-5.53055294e-02],
       [-1.51854195e-02],
       [-5.98950945e-02],
       [-1.37374267e-01],
       [-3.63355994e-01],
       [-3.54249030e-01],
       [-3.78998011e-01],
       [-4.86840844e-01],
       [-5.06314099e-01],
       [-5.3

In [201]:
def denormalize(data, original_data, type='min_max'):   ### default is min-max in this case(there is no reason behind this)

    if type == 'min_max':
        min = np.min(original_data)
        max = np.max(original_data)
        data = data * (max - min) + min
    elif type == 'z_score':
        mean = np.mean(original_data)
        std = np.std(original_data)
        data = data * std + mean
    elif type == 'max':                
        max = np.max(original_data)
        data = data * max
    else:
        raise ValueError("Invalid normalization type. Choose either 'max', 'min_max', or 'z_score'.")
    return data

In [202]:
Y_observ_unnorm = df_test['Qt']

In [203]:
Y_observ_unnorm = Y_observ_unnorm.to_numpy()

###  denormalized predicted discharge

In [204]:
### Standard denormalization (becasue we used standard(z-score) in this experiment)

Y_pred_denorm = denormalize(Y_pred, Y_observ_unnorm, type='z_score')

In [205]:
print(Y_pred_denorm)
print(Y_pred_denorm.shape)

[[ 12.755995 ]
 [ 10.233697 ]
 [ 10.684867 ]
 [  7.3210506]
 [  7.2298584]
 [  9.409645 ]
 [ 11.74454  ]
 [ 15.595984 ]
 [ 18.243553 ]
 [ 15.431513 ]
 [ 14.472126 ]
 [ 15.576697 ]
 [ 17.046093 ]
 [ 15.84177  ]
 [ 12.215141 ]
 [ 13.06384  ]
 [  9.932757 ]
 [  9.942653 ]
 [ 10.005505 ]
 [ 10.439261 ]
 [  9.911843 ]
 [  9.939628 ]
 [ 10.434923 ]
 [  7.0445557]
 [  8.909651 ]
 [ 15.6769   ]
 [ 15.544192 ]
 [ 22.010548 ]
 [ 26.155605 ]
 [ 23.766874 ]
 [ 24.9972   ]
 [ 23.626131 ]
 [ 21.250149 ]
 [ 14.320181 ]
 [ 14.599456 ]
 [ 13.840502 ]
 [ 10.533387 ]
 [  9.936219 ]
 [  9.207106 ]
 [  8.256474 ]
 [  8.737904 ]
 [  8.491665 ]
 [ 10.236871 ]
 [ 23.584007 ]
 [ 41.140602 ]
 [ 56.21416  ]
 [ 73.75468  ]
 [ 74.909256 ]
 [ 61.14579  ]
 [ 53.842846 ]
 [ 42.14463  ]
 [ 31.503082 ]
 [ 24.557686 ]
 [ 22.592316 ]
 [ 24.600191 ]
 [ 19.536728 ]
 [ 16.838379 ]
 [ 11.95928  ]
 [ 11.831688 ]
 [ 11.992695 ]
 [ 13.063084 ]
 [ 13.543808 ]
 [ 12.787529 ]
 [  7.2260876]
 [  5.42721  ]
 [ 12.026917 ]
 [ 52.8698

### denormalized Observed discharge (This is not necessary in this particular case!!!)
##### I can just use the 'Y_observ_unnorm'
#### Reason: Y_observ_unnorm = Y_pred have the same length of data (becasue the sequence used (1,1) for input and output sequence)

In [206]:
#### Instead use the following
Y_observ_unnorm = Y_observ_unnorm.reshape(547,1)
print(Y_observ_unnorm)

[[  8.209]
 [  7.878]
 [  7.716]
 [  7.406]
 [  7.547]
 [  9.705]
 [ 12.41 ]
 [ 11.4  ]
 [ 11.78 ]
 [ 11.18 ]
 [  9.592]
 [  8.971]
 [  8.767]
 [  8.476]
 [  9.1  ]
 [ 10.12 ]
 [ 10.23 ]
 [  8.742]
 [  8.002]
 [  7.598]
 [  7.39 ]
 [  7.179]
 [  6.955]
 [  7.089]
 [  7.843]
 [ 14.91 ]
 [ 13.15 ]
 [ 11.96 ]
 [ 17.44 ]
 [ 15.16 ]
 [ 12.77 ]
 [ 11.77 ]
 [ 11.33 ]
 [ 10.84 ]
 [ 10.46 ]
 [ 10.15 ]
 [ 10.13 ]
 [  9.378]
 [  8.771]
 [  8.122]
 [  7.523]
 [  7.336]
 [  8.263]
 [ 27.87 ]
 [ 29.08 ]
 [ 33.4  ]
 [ 53.18 ]
 [ 69.03 ]
 [ 54.06 ]
 [ 46.97 ]
 [ 38.65 ]
 [ 34.25 ]
 [ 27.97 ]
 [ 23.95 ]
 [ 21.63 ]
 [ 31.22 ]
 [ 31.49 ]
 [ 21.35 ]
 [ 17.45 ]
 [ 15.27 ]
 [ 13.7  ]
 [ 12.39 ]
 [ 11.52 ]
 [ 10.37 ]
 [  9.989]
 [ 21.   ]
 [ 37.43 ]
 [ 21.71 ]
 [ 16.06 ]
 [ 15.95 ]
 [ 16.09 ]
 [ 13.81 ]
 [ 12.27 ]
 [ 11.25 ]
 [ 11.7  ]
 [ 10.59 ]
 [ 11.45 ]
 [ 10.13 ]
 [  9.273]
 [  8.636]
 [  8.265]
 [  7.889]
 [  7.407]
 [  6.911]
 [  6.476]
 [  6.389]
 [  6.219]
 [  5.867]
 [  5.489]
 [  5.184]
 [  5.158]

# Plot the denomalized data

In [207]:
plt.figure(figsize = (16, 12))

plt.subplot(2, 1, 1)

plt.plot(Y_pred_denorm,'b', label = "Predicted")

plt.plot(Y_observ_unnorm , 'm',  label = "Observed")  

plt.legend()
plt.grid()

plt.subplot(2,1,2)

plt.plot(Y_pred_denorm - Y_observ_unnorm ,'r', label = "Error")

plt.legend()

plt.grid()

### NSE after denormalization

In [208]:
nse_d = he.evaluator(he.nse, Y_pred_denorm , Y_observ_unnorm )
print(nse_d)

[0.76058202]


### RMSE after denormalization (in m3/sec)

In [209]:
#### rmse in m3/sec
rmse_d = he.evaluator(he.rmse, Y_pred_denorm, Y_observ_unnorm)
print(rmse_d)    ### 

[15.00501252]


In [210]:
Y_pred_denorm

array([[ 12.755995 ],
       [ 10.233697 ],
       [ 10.684867 ],
       [  7.3210506],
       [  7.2298584],
       [  9.409645 ],
       [ 11.74454  ],
       [ 15.595984 ],
       [ 18.243553 ],
       [ 15.431513 ],
       [ 14.472126 ],
       [ 15.576697 ],
       [ 17.046093 ],
       [ 15.84177  ],
       [ 12.215141 ],
       [ 13.06384  ],
       [  9.932757 ],
       [  9.942653 ],
       [ 10.005505 ],
       [ 10.439261 ],
       [  9.911843 ],
       [  9.939628 ],
       [ 10.434923 ],
       [  7.0445557],
       [  8.909651 ],
       [ 15.6769   ],
       [ 15.544192 ],
       [ 22.010548 ],
       [ 26.155605 ],
       [ 23.766874 ],
       [ 24.9972   ],
       [ 23.626131 ],
       [ 21.250149 ],
       [ 14.320181 ],
       [ 14.599456 ],
       [ 13.840502 ],
       [ 10.533387 ],
       [  9.936219 ],
       [  9.207106 ],
       [  8.256474 ],
       [  8.737904 ],
       [  8.491665 ],
       [ 10.236871 ],
       [ 23.584007 ],
       [ 41.140602 ],
       [ 5

In [211]:
if (Y_pred_denorm < 0).any().any():
    print('The DataFrame contains negative values')

In [212]:
###logrmse
from sklearn.metrics import mean_squared_error
#### log of predicted values
y_pred_lg = np.log1p(Y_pred_denorm)

#### log of test values
y_test_lg = np.log1p(Y_observ_unnorm)

In [213]:
y_pred_lg

array([[2.6214747],
       [2.418918 ],
       [2.4582946],
       [2.1187885],
       [2.1077688],
       [2.342733 ],
       [2.545103 ],
       [2.8091607],
       [2.9571762],
       [2.799201 ],
       [2.7390401],
       [2.807998 ],
       [2.8929293],
       [2.823862 ],
       [2.5813632],
       [2.643607 ],
       [2.3917634],
       [2.3926682],
       [2.3983955],
       [2.4370515],
       [2.3898487],
       [2.3923917],
       [2.4366722],
       [2.0849955],
       [2.293509 ],
       [2.8140244],
       [2.8060353],
       [3.1359527],
       [3.3015835],
       [3.209507 ],
       [3.257989 ],
       [3.203808 ],
       [3.1023488],
       [2.729171 ],
       [2.747236 ],
       [2.69736  ],
       [2.445246 ],
       [2.39208  ],
       [2.323084 ],
       [2.2253232],
       [2.2760258],
       [2.2504141],
       [2.4192004],
       [3.2020962],
       [3.7410116],
       [4.0468016],
       [4.314212 ],
       [4.329539 ],
       [4.129483 ],
       [4.004472 ],


In [214]:
##calculate rmse
mse = mean_squared_error(y_pred_lg, y_test_lg)
rmse_lg = np.sqrt(mse)   
print(rmse_lg)

0.5532709903015924
